## Synthetic Data Creation and SQL insertion

The data generator produces a rich synthetic dataset with many-to-many relationships such as product-location pairs. 
The demo code requires you to have an environment (supplychain) that has pyodbc, urllib, sqlalchemy and azureml-defaults.
This Notebook performs the following actions:
1) Pre-Populates the Azure SQL Database sales and sales training tables with our synthetic data.
2) Pre-populates the Azure SQL Database parameters table for running inventory optimization simulations. 

In [1]:
import random
from datetime import datetime, timedelta
import numpy as np
import pyodbc
import sys
import urllib.request
import json
import os
import pandas as pd
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import random
import math
from sqlalchemy import create_engine,text, insert, update,Table

from azureml.core import Workspace, Datastore, Dataset, Experiment
from azureml.data.datapath import DataPath
 
 

from gluonts.dataset.common import load_datasets, ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.mx import DeepAREstimator
from gluonts.mx.distribution.gaussian import GaussianOutput
from gluonts.mx.trainer import Trainer
from gluonts.model.predictor import Predictor


from azure.ai.ml import command, Input, Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace = Workspace.from_config()



/anaconda/envs/supplychain/lib/python3.8/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


### Pre-Populate the Azure SQL Database sales and sales training tables with our synthetic data.

In [2]:
from enum import Enum

class SalesDataModel(Enum):
    RANDOM = 0
    SINUSOID = 1
    SPARSE = 2

from typing import List, Mapping

class SalesDataGenerator:
    def __init__(
        self,
        start_date: datetime,
        num_days: int,
        item_to_location: Mapping[str, List[str]],
        item_to_class: Mapping[str, List[str]],
        min_sales_per_day: int = 0,
        max_sales_per_day: int = 1000,
    ):
        self.start_date = start_date
        self.num_days = num_days
        self.item_to_location = item_to_location
        self.item_to_class = item_to_class
        self.min_sales_per_day = min_sales_per_day
        self.max_sales_per_day = max_sales_per_day

    def synthesize(self, data_model: SalesDataModel = SalesDataModel.RANDOM):
        sales_df = pd.DataFrame(columns=["Date", "Item", "Location", "Sales"])
        for item, locations in self.item_to_location.items():
            for location in locations:
                if data_model == SalesDataModel.RANDOM:
                    sales = np.random.randint(
                        self.min_sales_per_day, self.max_sales_per_day, self.num_days
                    )
                elif data_model == SalesDataModel.SINUSOID:
                    alpha = 0.9
                    gain = alpha * (self.max_sales_per_day - self.min_sales_per_day)
                    periods = np.random.randint(1, 6, 1)
                    nn = np.linspace(0, int(periods), self.num_days)
                    base_sales = (
                        gain * np.abs(np.cos(np.pi * nn))
                        + alpha * self.min_sales_per_day
                    )
                    noise = np.random.randint(
                        (1 - alpha) * self.min_sales_per_day,
                        (1 - alpha) * self.max_sales_per_day,
                        self.num_days,
                    )
                    sales = base_sales + noise
                    sales = sales.astype(int).tolist()
                elif data_model == SalesDataModel.SPARSE:
                    thresh = 0.1
                    sales = [
                        0
                        if np.random.rand() > thresh
                        else np.random.randint(
                            self.min_sales_per_day, self.max_sales_per_day, 1
                        )
                        for _ in range(self.num_days)
                    ]
                _sales_df = pd.DataFrame(
                    {
                        "Date": [
                            self.start_date + timedelta(days=diff)
                            for diff in range(self.num_days)
                        ],
                        "Item": [item] * self.num_days,
                        "Location": [location] * self.num_days,
                        "Sales": sales,
                        "Item_Class": [self.item_to_class[item]] * self.num_days,
                    }
                )
                sales_df = pd.concat([sales_df, _sales_df], ignore_index=True)
        
        # hnd, ord, bkd generation
        items = list(self.item_to_location.keys())
        qtyoh, cost, tons, unit_cost = [], [], [], []
        for item in items: 
            # QTYOH Randomly selected beween min_sales and max_sales
            _qtyoh = random.randint(
                min(0, self.min_sales_per_day), 
                self.max_sales_per_day
            )
            qtyoh.append(_qtyoh)
            # Tons is randomly chosen between 1 and 10 tons
            tons.append(
                _qtyoh * max(1.0, 10 * random.random())
            )
            # Cost is randomly selected bewteen 1 and 100
            _cost = round(max(1.0, 100 * random.random()), 2)
            cost.append(_cost)
            unit_cost.append(_cost / _qtyoh)

        hnd_df = pd.DataFrame(
            {
                "Item": items,
                "QTYOH": qtyoh,
                "Cost": cost, 
                "Tons": tons,
            }
        )
        
        # percentage randomly selected representing the percent of qtyoh that will be ordered
        coeff = round(random.random(), 2)
        # Each arrival is randomly selected over the sales generation window 
        arrival_dates = [
            self.start_date + timedelta(days=random.randint(0, self.num_days))
            for _ in range(len(items))
        ]
        ord_df = pd.DataFrame(
            {
                "Item": items, 
                "Total_Cost": [coeff * c for c in cost],
                "Tons": [coeff * t for t in tons],
                "Quantity": [coeff * q for q in qtyoh], 
                "Arrival_Date": arrival_dates,
            }
        )
        # shipping is chosen to be randomly between 2 and 5 days
        bkd_df = pd.DataFrame(
            {
                "Item": items, 
                "Ship_Date": [sd + timedelta(days=random.choice([2,3,4,5])) for sd in arrival_dates],
                "Qty_Remain": [(1-coeff) * q for q in qtyoh],
            }
        )

        return sales_df, hnd_df, ord_df, bkd_df
    
from typing import List, Mapping

class SalesDataGenerator:
    def __init__(
        self,
        start_date: datetime,
        num_days: int,
        item_to_location: Mapping[str, List[str]],
        item_to_class: Mapping[str, List[str]],
        min_sales_per_day: int = 0,
        max_sales_per_day: int = 1000,
    ):
        self.start_date = start_date
        self.num_days = num_days
        self.item_to_location = item_to_location
        self.item_to_class = item_to_class
        self.min_sales_per_day = min_sales_per_day
        self.max_sales_per_day = max_sales_per_day

    def synthesize(self, data_model: SalesDataModel = SalesDataModel.RANDOM):
        sales_df = pd.DataFrame(columns=["Date", "Item", "Location", "Sales"])
        for item, locations in self.item_to_location.items():
            for location in locations:
                if data_model == SalesDataModel.RANDOM:
                    sales = np.random.randint(
                        self.min_sales_per_day, self.max_sales_per_day, self.num_days
                    )
                elif data_model == SalesDataModel.SINUSOID:
                    alpha = 0.9
                    gain = alpha * (self.max_sales_per_day - self.min_sales_per_day)
                    periods = np.random.randint(1, 6, 1)
                    nn = np.linspace(0, int(periods), self.num_days)
                    base_sales = (
                        gain * np.abs(np.cos(np.pi * nn))
                        + alpha * self.min_sales_per_day
                    )
                    noise = np.random.randint(
                        (1 - alpha) * self.min_sales_per_day,
                        (1 - alpha) * self.max_sales_per_day,
                        self.num_days,
                    )
                    sales = base_sales + noise
                    sales = sales.astype(int).tolist()
                elif data_model == SalesDataModel.SPARSE:
                    thresh = 0.1
                    sales = [
                        0
                        if np.random.rand() > thresh
                        else np.random.randint(
                            self.min_sales_per_day, self.max_sales_per_day, 1
                        )
                        for _ in range(self.num_days)
                    ]
                _sales_df = pd.DataFrame(
                    {
                        "Date": [
                            self.start_date + timedelta(days=diff)
                            for diff in range(self.num_days)
                        ],
                        "Item": [item] * self.num_days,
                        "Location": [location] * self.num_days,
                        "Sales": sales,
                        "Item_Class": [self.item_to_class[item]] * self.num_days,
                    }
                )
                sales_df = pd.concat([sales_df, _sales_df], ignore_index=True)
        
        # hnd, ord, bkd generation
        items = list(self.item_to_location.keys())
        qtyoh, cost, tons, unit_cost = [], [], [], []
        for item in items: 
            # QTYOH Randomly selected beween min_sales and max_sales
            _qtyoh = random.randint(
                min(0, self.min_sales_per_day), 
                self.max_sales_per_day
            )
            qtyoh.append(_qtyoh)
            # Tons is randomly chosen between 1 and 10 tons
            tons.append(
                _qtyoh * max(1.0, 10 * random.random())
            )
            # Cost is randomly selected bewteen 1 and 100
            _cost = round(max(1.0, 100 * random.random()), 2)
            cost.append(_cost)
            unit_cost.append(_cost / _qtyoh)

        hnd_df = pd.DataFrame(
            {
                "Item": items,
                "QTYOH": qtyoh,
                "Cost": cost, 
                "Tons": tons,
            }
        )
        
        # percentage randomly selected representing the percent of qtyoh that will be ordered
        coeff = round(random.random(), 2)
        # Each arrival is randomly selected over the sales generation window 
        arrival_dates = [
            self.start_date + timedelta(days=random.randint(0, self.num_days))
            for _ in range(len(items))
        ]
        ord_df = pd.DataFrame(
            {
                "Item": items, 
                "Total_Cost": [coeff * c for c in cost],
                "Tons": [coeff * t for t in tons],
                "Quantity": [coeff * q for q in qtyoh], 
                "Arrival_Date": arrival_dates,
            }
        )
        # shipping is chosen to be randomly between 2 and 5 days
        bkd_df = pd.DataFrame(
            {
                "Item": items, 
                "Ship_Date": [sd + timedelta(days=random.choice([2,3,4,5])) for sd in arrival_dates],
                "Qty_Remain": [(1-coeff) * q for q in qtyoh],
            }
        )

        return sales_df, hnd_df, ord_df, bkd_df

For example, let's set the parameters to generate sales data over 5 years (`num_days`) backwards from today. We'll include 20 items, with each item being sold at 5 locations. Each item belongs to a item class randomly selected from 3 classes (this will be a static covariate when we get to the modeling section). 

In [4]:
num_days = 365 * 1
items = [f"Item_{idx}" for idx in range(20)]
locations = [f"Location_{idx}" for idx in range(5)]
item_to_class = {
    item: random.choice(["Item_Class_A", "Item_Class_B", "Item_Class_C"])
    for item in items
}

Next, we generate the data. For this example, we'll generate sinusoidal data (which also incorporates noise). Note that the `min_sales_per_day` argument is negative, meaning that the net sales for the day could be items returned. 

In [5]:
generator = SalesDataGenerator(
    start_date=datetime.today().date() - timedelta(days=num_days),
    num_days=num_days,
    item_to_location={item: locations for item in items},
    item_to_class=item_to_class,
    max_sales_per_day=1000,
    min_sales_per_day=-100,
)
sales_data, hnd_data, ord_data, bkd_data = generator.synthesize(data_model=SalesDataModel.SINUSOID)
ord_data.head()

With the dataset defined, we next need to preprocess it compatible with the model training object. To do this, we create a list of dictionaries with specific keys. Note that we are scaling the data in this preprocessing step to be compatable with the statistical structure of the model we define later. 

Save the above datasets into new database inside SQL database: we need to make the tables smaller because we create a huge dataset

In [48]:
num_rows = 1000
col_names = list(sales_data.columns)
sales_data_small = sales_data.loc[0:num_rows,col_names]
sales_data_small.shape

def get_engine():
    server = 'tcp:sql-sa-htt2.database.windows.net' 
    database = 'supplychain'
    username = 'sasqladmin' 
    password = 'SA-G10rg10-!$!$' 

    quoted = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

    return engine
engine=get_engine()


try:
    sales_data_small.astype(str).to_sql('sc_sales_data', if_exists='append', schema='dbo', con = engine, index=False)
    print("Persisted Sales Data into the DB")
except:
    print("failed to persist item {} to the DB".format(sales_data['Item']))
    print("data ", sales_data_small)





failed to persist item 0       Item_0
1       Item_0
2       Item_0
3       Item_0
4       Item_0
         ...  
2695    Item_4
2696    Item_4
2697    Item_4
2698    Item_4
2699    Item_4
Name: Item, Length: 2700, dtype: object to the DB
data        Unnamed: 0        Date    Item    Location  Sales    Item_Class
0              0  2022-05-25  Item_0  Location_0    936  Item_Class_B
1              1  2022-05-26  Item_0  Location_0    942  Item_Class_B
2              2  2022-05-27  Item_0  Location_0    893  Item_Class_B
3              3  2022-05-28  Item_0  Location_0    931  Item_Class_B
4              4  2022-05-29  Item_0  Location_0    901  Item_Class_B
...          ...         ...     ...         ...    ...           ...
2695        2695  2022-11-16  Item_4  Location_2    888  Item_Class_A
2696        2696  2022-11-17  Item_4  Location_2    962  Item_Class_A
2697        2697  2022-11-18  Item_4  Location_2    988  Item_Class_A
2698        2698  2022-11-19  Item_4  Location_2    932 

## Prepopulate the Parameters Table Here

In [18]:

##Populate the params table DO THIS ONLY ONCE

def get_engine():
    server = 'tcp:sql-sa-htt2.database.windows.net' 
    database = 'supplychain'
    username = 'sasqladmin' 
    password = 'SA-G10rg10-!$!$' 

    quoted = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

    return engine

# Quick overview of this function, it picks up the new datasets and puts in the format that you can generate the new demand forecast and
# the input format for the optimization function
def create_ensemble_test_ds(sales_data):
    items = [f"Item_{idx}" for idx in range(5)]
    item_to_class = {
        item: random.choice(["Item_Class_A", "Item_Class_B", "Item_Class_C"])
        for item in items
    } 
    #print(item_to_class)

    ## create variables
    start = sales_data.Date.min()
    end = sales_data.Date.max()
    dates = list(pd.date_range(start, end))

    # create ensemble
    ensemble = []

    for item in set(sales_data.Item): 
        for location in set(sales_data.Location):
            sales = sales_data[(sales_data.Item == item) & (sales_data.Location == location)].Sales.values
            if len(sales): 
                scale = max(abs(sales))
                ensemble.append(
                    {
                        "class": item_to_class[item],
                        "item": item,
                        "sales": sales,
                        "start": start,
                        "end": end,
                        "total": sum(sales),
                        "scale": scale,
                        "location": location,
                        "sales_scaled": sales / scale,
                    }
                )
    ensemble = sorted(ensemble, key=lambda k: k["total"], reverse=True)
    # create test ds 
    static_cats = [(e["class"].strip(), e["item"].strip(), e["location"].strip()) for e in ensemble]
    test_target_values = np.array([np.array(e["sales_scaled"]) for e in ensemble])
    item_to_index = {item: idx for idx, item in enumerate(set([sc[0] for sc in static_cats]))}
    item_class_to_index = {item: idx for idx, item in enumerate(set([sc[1] for sc in static_cats]))}
    location_to_index = {loc: idx for idx, loc in enumerate(set([sc[2] for sc in static_cats]))}

    test_ds = ListDataset([
        {
            FieldName.TARGET: target,
            FieldName.START: start,
            FieldName.FEAT_STATIC_CAT: [
                item_to_index[sc_item], 
                item_class_to_index[sc_item_class], 
                location_to_index[sc_location]
            ],
        }
        for target, (sc_item, sc_item_class, sc_location) in zip(test_target_values, static_cats)
    ], freq="D")

    return ensemble, test_ds 

def generate_forecasts(dataset, predictor, num_samples):
        
        forecast_it, _ = make_evaluation_predictions(dataset=dataset, predictor=predictor, num_samples=num_samples)
        forecasts = [f for f in forecast_it]

        return forecasts




dataset = Dataset.get_by_name(workspace, name='training_sales_data_2', version='latest')
sales_data = dataset.to_pandas_dataframe() 
ensemble,test_ds = create_ensemble_test_ds(sales_data)
df_params = pd.DataFrame(ensemble)

df_params.drop(['sales','start','end','total','scale','sales_scaled'],axis=1,inplace=True)

df_params['Inventory_Capacity']= " "
df_params['Max_Quantity']=" "
df_params['Holding_Cost']=" "
df_params['Order_Cost']=" "
df_params['Lead_Time']=" "
df_params['Backlog_Cost']=" "
df_params['Max_Backlogged']=" "
df_params['Initial_Inventory']=" "


                
invCap = 1000
maxQ =  1000
orderCost = 500
holdCost =  2 
leadTime =  10 
backLogCost =  10 
maxBackLogged = 1000
leadTime = 10
initInv = 500

for i in range(len(df_params)):

    npar = random.randint(i, 100)
    
    df_params.iloc[i, 3] = invCap + npar
    df_params.iloc[i, 4] = maxQ + npar
    df_params.iloc[i, 5] = holdCost + npar
    df_params.iloc[i, 6] = orderCost + npar
    df_params.iloc[i, 7] = leadTime + npar
    df_params.iloc[i, 8] = backLogCost + npar
    df_params.iloc[i, 9] = maxBackLogged + npar
    df_params.iloc[i, 10] = initInv + npar





Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


In [19]:
df_params.sort_values(by=['class','item','location']).head(50)


,class,item,location,Inventory_Capacity,Max_Quantity,Holding_Cost,Order_Cost,Lead_Time,Backlog_Cost,Max_Backlogged,Initial_Inventory
12,Item_Class_B,Item_0,Location_0,1071,1071,73,571,81,81,1071,571
0,Item_Class_B,Item_0,Location_1,1057,1057,59,557,67,67,1057,557
8,Item_Class_B,Item_0,Location_2,1100,1100,102,600,110,110,1100,600
3,Item_Class_B,Item_1,Location_0,1020,1020,22,520,30,30,1020,520
2,Item_Class_B,Item_1,Location_1,1033,1033,35,533,43,43,1033,533
5,Item_Class_B,Item_1,Location_2,1061,1061,63,561,71,71,1061,561
6,Item_Class_B,Item_2,Location_0,1030,1030,32,530,40,40,1030,530
1,Item_Class_B,Item_2,Location_1,1076,1076,78,576,86,86,1076,576
9,Item_Class_B,Item_2,Location_2,1099,1099,101,599,109,109,1099,599
13,Item_Class_C,Item_3,Location_0,1016,1016,18,516,26,26,1016,516


In [20]:
engine = get_engine()

try:
    df_params.astype(str).to_sql('sc_params_table', if_exists='append', schema='dbo', con = engine,index_label=None)
    print("Persisted Parameters Table successfully to the DB")
except pyodbc.Error as err: 
    raise err

Persisted Parameters Table successfully to the DB
